In [2]:
import pandas as pd
import numpy as np

In [12]:
import os
import requests
from PIL import Image
from io import BytesIO

# Directory to save images
save_dir = r"D:\downloads\ImageComp\PoC"
os.makedirs(save_dir, exist_ok=True)

# Filter tags
required_tags = {'Facade', 'Room', 'Outdoors', 'Lobby/Common Area'}

# Simulated DataFrame (replace this with your actual data)

#First line doesn't work
data = {
    'image_info': [
        [{'media_tag': 'Swimming Pool', 'url': 'https://gos3.ibcdn.com/af85276f-30a8-4eae-a1c4-0a5cba0d6a5a.png', 'mmt_tags': ['Swimming Pool']},
         {'media_tag': 'Room', 'url': 'https://gos3.ibcdn.com/42a45b5ecea311eb9fb80242ac11000d.jpg', 'mmt_tags': ['Room']},
         {'media_tag': 'Facade', 'url': 'https://gos3.ibcdn.com/9cea645ccea111eb917b0242ac11000d.jfif', 'mmt_tags': ['Facade']},
         {'media_tag': 'Outdoors', 'url': 'https://gos3.ibcdn.com/af85276f-30a8-4eae-a1c4-0a5cba0d6a5a.png', 'mmt_tags': ['Outdoors']}]
    ]
}

# Extract the first list of image info
all_image_info = data['image_info'][0]

# Filter images based on required tags
filtered_images = [
    item for item in all_image_info
    if any(tag in required_tags for tag in item.get('mmt_tags', []))
]

# Download and process images
for image in filtered_images:
    url = image.get('url')
    if not url:
        continue

    try:
        # Log URL to ensure it is being processed
        print(f"Processing URL: {url}")

        # Download image
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Check if the request was successful

        # Open the image
        img = Image.open(BytesIO(response.content))

        # Resize to 2400x1600
        img_resized = img.resize((2400, 1600), Image.ANTIALIAS)

        # Handle image format to save as JPEG
        original_name = os.path.basename(url).split('.')[0]
        file_name = original_name + ".jpg"
        save_path = os.path.join(save_dir, file_name)

        # Convert to RGB (if not already) and save as .jpg
        img_resized.convert("RGB").save(save_path, "JPEG")
        print(f"Downloaded and saved: {save_path}")
        
    except Exception as e:
        print(f"Failed to process {url}: {e}")


Processing URL: https://gos3.ibcdn.com/42a45b5ecea311eb9fb80242ac11000d.jpg


C:\Users\mmt9823\AppData\Local\Temp\ipykernel_25436\3368391535.py:50: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = img.resize((2400, 1600), Image.ANTIALIAS)


Downloaded and saved: D:\downloads\ImageComp\PoC\42a45b5ecea311eb9fb80242ac11000d.jpg
Processing URL: https://gos3.ibcdn.com/9cea645ccea111eb917b0242ac11000d.jfif
Downloaded and saved: D:\downloads\ImageComp\PoC\9cea645ccea111eb917b0242ac11000d.jpg
Processing URL: https://gos3.ibcdn.com/af85276f-30a8-4eae-a1c4-0a5cba0d6a5a.png
Downloaded and saved: D:\downloads\ImageComp\PoC\af85276f-30a8-4eae-a1c4-0a5cba0d6a5a.jpg


In [2]:
import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO

# Paths and required tags
excel_path = r"D:\downloads\ImageComp\PoC\image_urls_with_mmt_htl_id.xlsx"
save_base_dir = r"D:\downloads\ImageComp\PoC"
required_tags = {'Facade', 'Room', 'Outdoors', 'Lobby/Common Area'}

# Load data from Excel
sheet_name = "Sheet1"
df = pd.read_excel(excel_path, sheet_name=sheet_name)

# Ensure required columns exist
required_columns = ['MMT Hotel ID', 'URL', 'MMT Tags']
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"One or more required columns {required_columns} are missing from the sheet.")

# Process the data
for _, row in df.iterrows():
    hotel_id = row['MMT Hotel ID']
    url = row['URL']
    tags = row['MMT Tags']
    
    # Ensure valid data
    if not hotel_id or not url or pd.isna(tags):
        print(f"Skipping row with missing data: Hotel ID={hotel_id}, URL={url}, Tags={tags}")
        continue

    # Parse tags (assuming tags are stored as comma-separated values)
    tag_list = [tag.strip() for tag in tags.split(',')]
    
    # Check if any tag matches the required tags
    if not set(tag_list).intersection(required_tags):
        print(f"Skipping URL due to unmatched tags: {url}")
        continue

    try:
        # Create a folder for the hotel ID
        hotel_dir = os.path.join(save_base_dir, str(hotel_id))
        os.makedirs(hotel_dir, exist_ok=True)

        # Download the image
        print(f"Processing URL: {url}")
        response = requests.get(url, stream=True, timeout=10)
        response.raise_for_status()  # Raise an error for bad status

        # Open and resize the image
        img = Image.open(BytesIO(response.content))
        img_resized = img.resize((2400, 1600), Image.ANTIALIAS)

        # Save the image as JPEG
        original_name = os.path.splitext(os.path.basename(url))[0]
        file_name = f"{original_name}.jpg"
        save_path = os.path.join(hotel_dir, file_name)
        img_resized.convert("RGB").save(save_path, "JPEG")

        print(f"Downloaded and saved: {save_path}")

    except Exception as e:
        print(f"Failed to process URL: {url}. Error: {e}")


Processing URL: https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/room-imgs/202109151802278551-2312-b715c87a161911eca38a0a58a9feac02.jpg


C:\Users\mmt9823\AppData\Local\Temp\ipykernel_10580\3825578615.py:52: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = img.resize((2400, 1600), Image.ANTIALIAS)


Downloaded and saved: D:\downloads\ImageComp\PoC\202109151802278551\202109151802278551-2312-b715c87a161911eca38a0a58a9feac02.jpg
Processing URL: https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-eea89232161811eca5570a58a9feac02.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\202109151802278551\202109151802278551-eea89232161811eca5570a58a9feac02.jpg
Skipping URL due to unmatched tags: https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-8f51a0bc161811ecb6fd0a58a9feac02.jpg
Skipping URL due to unmatched tags: https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-48b37662161811ecb5760a58a9feac02.jpg
Processing URL: https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-6860cf14161811ecb6fd0a58a9feac02.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\202109151802278551\202109151802278551-6860cf14161811ecb6fd0a58a9feac02.jpg
Skipping URL due to unmatched tags: https://r2imghtlak.mmtcdn.com/r2-mm

Downloaded and saved: D:\downloads\ImageComp\PoC\201004211332365958\6f282c229f7511eaa6240242ac110002.jpg
Skipping URL due to unmatched tags: https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/201004211332365958-6db0710cb65111ec9b500a58a9feac02.jpg
Processing URL: https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/201004211332365958-c9a3f928b64411eca0330a58a9feac02.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\201004211332365958\201004211332365958-c9a3f928b64411eca0330a58a9feac02.jpg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/bdf0c1ca57c111ebaa120242ac110002.jpg
Skipping URL due to unmatched tags: https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/201004211332365958-e44acffcb65011ecb3d50a58a9feac02.jpg
Processing URL: https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/201004211332365958-db0dde64b64d11ec9b500a58a9feac02.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\201004211332365958\201004211332365958-db0dde64b64d11ec9b500a58a9feac02.jpg

Downloaded and saved: D:\downloads\ImageComp\PoC\201004211332365958\0fc07670b70d11ecae1c0a58a9feac02.jpg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/34ec7d68b70d11ec91070a58a9feac02.jpg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/77o1urue8d59vfj66a6pk7pc0018.jpg
Processing URL: https://gos3.ibcdn.com/e4d67eec352511e6b9a40015c5f4277e.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\201004211332365958\e4d67eec352511e6b9a40015c5f4277e.jpg
Processing URL: https://gos3.ibcdn.com/bccce6ae7dc011e7b9c6025f77df004f.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\201004211332365958\bccce6ae7dc011e7b9c6025f77df004f.jpg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/9cb9034c99f011ecae430a58a9feac02.jpg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/a366d36899f011ec93880a58a9feac02.jpg
Processing URL: https://gos3.ibcdn.com/e8bb6d2ccc4c11e8933e0a9df65c8753.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\201004211332365958\e8bb6d2cc

...

Downloaded and saved: D:\downloads\ImageComp\PoC\202008232345579694\1e350992428f11ed9b9a0a58a9feac02.jpg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/1daaa518428f11ed858c0a58a9feac02.jpg
Processing URL: https://gos3.ibcdn.com/1e0a6e1c428f11edb8e30a58a9feac02.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\202008232345579694\1e0a6e1c428f11edb8e30a58a9feac02.jpg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/1b3092a2428f11edb25d0a58a9feac02.jpg
Skipping URL due to unmatched tags: https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/201708081807157535-1e9b830e660811eaa56e0242ac11000a.jpg
Processing URL: https://gos3.ibcdn.com/21b01ea06c6f11e7b07a02755708f0b3.jfif
Downloaded and saved: D:\downloads\ImageComp\PoC\201708081807157535\21b01ea06c6f11e7b07a02755708f0b3.jpg
Processing URL: https://gos3.ibcdn.com/1a328dd26d2f11e789680a209fbd0127.jfif
Downloaded and saved: D:\downloads\ImageComp\PoC\201708081807157535\1a328dd26d2f11e789680a209fbd0127.jpg
Skipping U

Downloaded and saved: D:\downloads\ImageComp\PoC\201708081807157535\274954746d2f11e7bc590a9df65c8753.jpg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/2ec9b16c6d2f11e7a7210a9df65c8753.jfif
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/33d9e7b650ea11eab8e30242ac110003.jfif
Processing URL: https://gos3.ibcdn.com/2912d6d8954611ec92f50a58a9feac02.jpeg
Downloaded and saved: D:\downloads\ImageComp\PoC\201704231638138752\2912d6d8954611ec92f50a58a9feac02.jpg
Processing URL: https://gos3.ibcdn.com/9652b8cc954511ecaf830a58a9feac02.jpeg
Downloaded and saved: D:\downloads\ImageComp\PoC\201704231638138752\9652b8cc954511ecaf830a58a9feac02.jpg
Processing URL: https://gos3.ibcdn.com/2912580c954611ec9f000a58a9feac02.jpeg
Downloaded and saved: D:\downloads\ImageComp\PoC\201704231638138752\2912580c954611ec9f000a58a9feac02.jpg
Processing URL: https://gos3.ibcdn.com/5e2f3114954511ecaf830a58a9feac02.jpeg
Downloaded and saved: D:\downloads\ImageComp\PoC\201704231638138752\5e2f311495

Downloaded and saved: D:\downloads\ImageComp\PoC\201704231638138752\3e4935ac954a11ecb8f70a58a9feac02.jpg
Processing URL: https://gos3.ibcdn.com/3e047d90954a11ec8ad70a58a9feac02.jpeg
Downloaded and saved: D:\downloads\ImageComp\PoC\201704231638138752\3e047d90954a11ec8ad70a58a9feac02.jpg
Processing URL: https://gos3.ibcdn.com/6fb7787e954a11ecbb890a58a9feac02.jpeg
Downloaded and saved: D:\downloads\ImageComp\PoC\201704231638138752\6fb7787e954a11ecbb890a58a9feac02.jpg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/6f0b2c72954a11ecafcf0a58a9feac02.jpeg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/6efd3b58954a11ecad920a58a9feac02.jpeg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/d58b6f02954a11ec84b00a58a9feac02.jpeg
Processing URL: https://gos3.ibcdn.com/ba49b36a954b11ecbb890a58a9feac02.jpeg
Downloaded and saved: D:\downloads\ImageComp\PoC\201704231638138752\ba49b36a954b11ecbb890a58a9feac02.jpg
Skipping URL due to unmatched tags: https://r2imghtlak.mmt

Downloaded and saved: D:\downloads\ImageComp\PoC\201709081331037533\201709081331037533-30069-7b92a8be1f6a11eb9f070242ac110003.jpg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/8455bf566cee11eab8f00242ac110008.jpg
Processing URL: https://gos3.ibcdn.com/8578d83c6cee11eaa91f0242ac110002.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\201709081331037533\8578d83c6cee11eaa91f0242ac110002.jpg
Processing URL: https://gos3.ibcdn.com/4c8b545cae8411edb2a70a58a9feac02.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\201709081331037533\4c8b545cae8411edb2a70a58a9feac02.jpg
Processing URL: https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/room-imgs/201709081331037533-4791976-39d20cd214cf11eeab5f0a58a9feac02.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\201709081331037533\201709081331037533-4791976-39d20cd214cf11eeab5f0a58a9feac02.jpg
Processing URL: https://gos3.ibcdn.com/74d6c9ea45c811ebbc2c0242ac110002.jpeg
Downloaded and saved: D:\downloads\ImageComp\PoC\20170908133103753

Downloaded and saved: D:\downloads\ImageComp\PoC\201709081331037533\7494ed9a173511ee9f650a58a9feac02.jpg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/750c9930173511eeba430a58a9feac02.jpg
Skipping URL due to unmatched tags: https://gos3.ibcdn.com/751acdf2173511eebdcd0a58a9feac02.jpg
Processing URL: https://gos3.ibcdn.com/80c87b2c173511eeb5670a58a9feac02.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\201709081331037533\80c87b2c173511eeb5670a58a9feac02.jpg
Processing URL: https://gos3.ibcdn.com/81052cca173511eeb7a30a58a9feac02.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\201709081331037533\81052cca173511eeb7a30a58a9feac02.jpg
Processing URL: https://gos3.ibcdn.com/80b44b7a173511ee8cc40a58a9feac02.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\201709081331037533\80b44b7a173511ee8cc40a58a9feac02.jpg
Processing URL: https://gos3.ibcdn.com/8c00722e173511eebe6d0a58a9feac02.jpg
Downloaded and saved: D:\downloads\ImageComp\PoC\201709081331037533\8c00722e173511ee